# PRL and eye-tracking 

In [163]:
import os
import pandas as pd
import numpy as np
from functools import reduce
from pandas.api.types import CategoricalDtype
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
import sys
import datetime


In [164]:
now = datetime.datetime.now()
formatted_date = now.strftime("%Y-%m-%d %H:%M:%S")

print("Written by Corrado Caudek")
print("Last change:", formatted_date)

Written by Corrado Caudek
Last change: 2023-03-18 09:40:02


In [165]:
%run helpers_cc_prlet.ipynb

In [166]:
%pwd

'/Users/corrado/_repositories/prl_eye_movements/scripts'

Save the folder names contained in the directory `thesis_marta`.

In [167]:
# Set the directory path
path_to_project_folder = os.path.join(
        "..",
        "data",
        "raw",
        "thesis_marta"
    )
print(path_to_project_folder)

../data/raw/thesis_marta


Get the names of the subjects folders.

In [168]:
names_of_subjects_folders = get_names_of_subj_folders(path_to_project_folder)
print(names_of_subjects_folders)

['fr_va_1960_08_18_538_u', 'ma_va_1998_07_04_538_d']


Select one subject.

In [169]:
# SUBJECT_INDEX identifies the elements of the `folder` list.
SUBJECT_INDEX = 1

For the selected subject, get the name of the file in the `biographical_data` folder.

In [170]:
bio_data_df = get_biographic_data(SUBJECT_INDEX, names_of_subjects_folders)

subject_id_string = bio_data_df["subject_id_string"]
subject_id_number = bio_data_df["subject_id_number"]

Read thresholds from calibration data.

In [171]:
thresholds = get_thresholds(SUBJECT_INDEX, names_of_subjects_folders)
print(thresholds)

[0.53, 0.57, 0.58, 0.5]


Read eye-tracking data.

In [172]:
fix_df = get_eye_tracking_data(SUBJECT_INDEX, names_of_subjects_folders)

In [173]:
fix_df.head()

,prop_dx_fixation,prop_sx_fixation,prop_center_fixation,block,trial,stim_left,stim_left_img_number,stim_right,stim_right_img_number,subject_id_number2
0,0.750000,0.000000,0.250000,5,107,orange,9,blue,4,0GLB5VNRFDP
1,0.692308,0.000000,0.307692,5,102,blue,5,orange,9,0GLB5VNRFDP
2,0.531915,0.212766,0.255319,1,14,blue,3,orange,7,0GLB5VNRFDP
3,0.531250,0.015625,0.453125,2,44,blue,7,orange,5,0GLB5VNRFDP
4,0.818182,0.000000,0.181818,3,74,orange,2,blue,5,0GLB5VNRFDP


In [174]:
# calculate mean of prop_dx_fixation column
mean_dx = fix_df['prop_dx_fixation'].mean()

# calculate mean of prop_sx_fixation column
mean_sx = fix_df['prop_sx_fixation'].mean()

# calculate mean of prop_center_fixation column
mean_center = fix_df['prop_center_fixation'].mean()

[mean_dx, mean_sx, mean_center]

[0.6447559720112035, 0.0747397664170781, 0.2805042615717185]

Read PRL data

In [175]:
prl_df = read_prl_data(SUBJECT_INDEX, names_of_subjects_folders)


Merge the fixation DataFrame and the PRL DataFrame

In [176]:
tot_df = pd.merge(fix_df, prl_df, on="trial", how="outer")
tot_df.columns


Index(['prop_dx_fixation', 'prop_sx_fixation', 'prop_center_fixation', 'block',
       'trial', 'stim_left', 'stim_left_img_number', 'stim_right',
       'stim_right_img_number', 'subject_id_number2', 'more_rewarded_color',
       'stimulus_choice', 'blue_position', 'orange_position', 'left_img_name',
       'right_img_name', 'location_choice', 'keyboard_input', 'orange_reward',
       'blue_reward', 'outcome', 'start_reaction_time', 'end_reaction_time',
       'start_absolute_time', 'end_absolute_time'],
      dtype='object')

In [177]:
# Calculate rt column
tot_df["rt"] = tot_df["end_reaction_time"] - tot_df["start_reaction_time"]

# Create is_negative_img_chosen column
tot_df["is_negative_img_chosen"] = tot_df["stimulus_choice"].apply(
    lambda x: 1 if x == "blue" else 0
)

# Create feedback column
tot_df["feedback"] = tot_df["outcome"].apply(lambda x: 1 if x == "euro" else 0)

# Calculate mean feedback
mean_feedback = tot_df["feedback"].mean()
mean_feedback


0.72

In [178]:
tot_df.left_img_name

0      orange_9
1        blue_5
2        blue_3
3        blue_7
4      orange_2
         ...   
145      blue_3
146      blue_6
147    orange_6
148    orange_8
149    orange_1
Name: left_img_name, Length: 150, dtype: object

Recode `left_img_name` (orange_5, blue_8, ...) so as to keep only the name (discard the number).

In [179]:
left_img = tot_df["left_img_name"].str.split("_")
tot_df["image_on_left"] = pd.DataFrame(left_img.tolist())[0]

right_img = tot_df["right_img_name"].str.split("_")
tot_df["image_on_right"] = pd.DataFrame(right_img.tolist())[0]

In [180]:
tot_df.groupby(["image_on_left", "image_on_right"]).agg(
    avg_left_fix=("prop_sx_fixation", "mean"),
    avg_right_fix=("prop_dx_fixation", "mean"),
)


,,avg_left_fix,avg_right_fix
image_on_left,image_on_right,,
blue,orange,0.120768,0.508559
orange,blue,0.023525,0.796299


In [181]:
tot_df["is_preference_right"] = tot_df["prop_dx_fixation"] - tot_df["prop_sx_fixation"]

fm = smf.ols("is_preference_right ~ stim_right", data=tot_df).fit()
print(fm.summary())

                             OLS Regression Results                            
Dep. Variable:     is_preference_right   R-squared:                       0.292
Model:                             OLS   Adj. R-squared:                  0.288
Method:                  Least Squares   F-statistic:                     61.17
Date:                 Sat, 18 Mar 2023   Prob (F-statistic):           9.10e-13
Time:                         09:40:55   Log-Likelihood:                -31.741
No. Observations:                  150   AIC:                             67.48
Df Residuals:                      148   BIC:                             73.50
Df Model:                            1                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             

In [182]:
tot_df.groupby(["location_choice"]).agg(
    avg_left=("prop_sx_fixation", "mean"), avg_right=("prop_dx_fixation", "mean")
)

,avg_left,avg_right
location_choice,,
left,0.09229,0.602937
right,0.05719,0.686575


Average positive feedback.

In [183]:
mean_feedback = tot_df["feedback"].mean()
print(mean_feedback)

0.72
